In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hotdognothotdog-cnn/hotdog-nothotdog/test/hotdog/hotdog (99).jpg
/kaggle/input/hotdognothotdog-cnn/hotdog-nothotdog/test/hotdog/hotdog (107).jpg
/kaggle/input/hotdognothotdog-cnn/hotdog-nothotdog/test/hotdog/hotdog (15).jpg
/kaggle/input/hotdognothotdog-cnn/hotdog-nothotdog/test/hotdog/hotdog (30).jpg
/kaggle/input/hotdognothotdog-cnn/hotdog-nothotdog/test/hotdog/hotdog (227).jpg
/kaggle/input/hotdognothotdog-cnn/hotdog-nothotdog/test/hotdog/hotdog (277).jpg
/kaggle/input/hotdognothotdog-cnn/hotdog-nothotdog/test/hotdog/hotdog (168).jpg
/kaggle/input/hotdognothotdog-cnn/hotdog-nothotdog/test/hotdog/hotdog (14).jpg
/kaggle/input/hotdognothotdog-cnn/hotdog-nothotdog/test/hotdog/hotdog (183).jpg
/kaggle/input/hotdognothotdog-cnn/hotdog-nothotdog/test/hotdog/hotdog (189).jpg
/kaggle/input/hotdognothotdog-cnn/hotdog-nothotdog/test/hotdog/hotdog (281).jpg
/kaggle/input/hotdognothotdog-cnn/hotdog-nothotdog/test/hotdog/hotdog (108).jpg
/kaggle/input/hotdognothotdog-cnn/hotdog-not

/kaggle/input/hotdognothotdog-cnn/hotdog-nothotdog/train/hotdog/hotdog (1385).jpg
/kaggle/input/hotdognothotdog-cnn/hotdog-nothotdog/train/hotdog/hotdog (99).jpg
/kaggle/input/hotdognothotdog-cnn/hotdog-nothotdog/train/hotdog/hotdog (839).jpg
/kaggle/input/hotdognothotdog-cnn/hotdog-nothotdog/train/hotdog/hotdog (1305).jpg
/kaggle/input/hotdognothotdog-cnn/hotdog-nothotdog/train/hotdog/hotdog (107).jpg
/kaggle/input/hotdognothotdog-cnn/hotdog-nothotdog/train/hotdog/hotdog (759).jpg
/kaggle/input/hotdognothotdog-cnn/hotdog-nothotdog/train/hotdog/hotdog (555).jpg
/kaggle/input/hotdognothotdog-cnn/hotdog-nothotdog/train/hotdog/hotdog (1412).jpg
/kaggle/input/hotdognothotdog-cnn/hotdog-nothotdog/train/hotdog/hotdog (1025).jpg
/kaggle/input/hotdognothotdog-cnn/hotdog-nothotdog/train/hotdog/hotdog (1249).jpg
/kaggle/input/hotdognothotdog-cnn/hotdog-nothotdog/train/hotdog/hotdog (15).jpg
/kaggle/input/hotdognothotdog-cnn/hotdog-nothotdog/train/hotdog/hotdog (1495).jpg
/kaggle/input/hotdognoth

/kaggle/input/hotdognothotdog-cnn/hotdog-nothotdog/train/nothotdog/nothotdog (145).jpg
/kaggle/input/hotdognothotdog-cnn/hotdog-nothotdog/train/nothotdog/nothotdog (279).jpg
/kaggle/input/hotdognothotdog-cnn/hotdog-nothotdog/train/nothotdog/nothotdog (593).jpg
/kaggle/input/hotdognothotdog-cnn/hotdog-nothotdog/train/nothotdog/nothotdog (794).jpg
/kaggle/input/hotdognothotdog-cnn/hotdog-nothotdog/train/nothotdog/nothotdog (526).jpg
/kaggle/input/hotdognothotdog-cnn/hotdog-nothotdog/train/nothotdog/nothotdog (475).jpg
/kaggle/input/hotdognothotdog-cnn/hotdog-nothotdog/train/nothotdog/nothotdog (78).jpg
/kaggle/input/hotdognothotdog-cnn/hotdog-nothotdog/train/nothotdog/nothotdog (339).jpg
/kaggle/input/hotdognothotdog-cnn/hotdog-nothotdog/train/nothotdog/nothotdog (296).jpg
/kaggle/input/hotdognothotdog-cnn/hotdog-nothotdog/train/nothotdog/nothotdog (188).jpg
/kaggle/input/hotdognothotdog-cnn/hotdog-nothotdog/train/nothotdog/nothotdog (1212).jpg
/kaggle/input/hotdognothotdog-cnn/hotdog-no

In [2]:
import os 
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 
from PIL import Image
import glob
%matplotlib inline

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from  IPython.display import display
import plotly.express as px

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, experimental, MaxPool2D, BatchNormalization
from tensorflow.keras.losses import sparse_categorical_crossentropy, binary_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau 
from tensorflow.data import Dataset
from tensorflow.keras import Input, Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.random import set_seed
from tensorflow.keras.utils import to_categorical
from tensorflow import test
import random

np.random.seed(42)
set_seed(42)
random.seed(42)

In [3]:
files = glob.glob("/kaggle/input/hotdognothotdog-cnn/hotdog-nothotdog/train/hotdog/*.jpg")
TrainData = []
index = 0

for file in files:
    img = Image.open(file)
    img = img.resize((128, 128))
    imgarray = np.asarray(img)
    TrainData.append((imgarray,1))

files = glob.glob("/kaggle/input/hotdognothotdog-cnn/hotdog-nothotdog/train/nothotdog/*.jpg")
index = 0
for file in files:
    img = Image.open(file)
    img = img.resize((128, 128))
    imgarray = np.asarray(img)
    TrainData.append((imgarray,0))

print("comp")

random.shuffle(TrainData)

comp


In [4]:
files = glob.glob("/kaggle/input/hotdognothotdog-cnn/hotdog-nothotdog/test/hotdog/*.jpg")
TestData = []
index = 0

for file in files:
    img = Image.open(file)
    img = img.resize((128, 128))
    imgarray = np.asarray(img)
    TestData.append((imgarray,1))

files = glob.glob("/kaggle/input/hotdognothotdog-cnn/hotdog-nothotdog/test/nothotdog/*.jpg")
index = 0
for file in files:
    img = Image.open(file)
    img = img.resize((128, 128))
    imgarray = np.asarray(img)
    TestData.append((imgarray,0))

print("comp")

random.shuffle(TestData)

comp


In [5]:
Train_X = np.array([item[0] for item in TrainData])
Train_Y = np.array([item[1] for item in TrainData])

Test_X = np.array([item[0] for item in TestData])
Test_Y = np.array([item[1] for item in TestData])

Train_X = Train_X.astype('float32')
Test_X = Test_X.astype('float32')

Train_X, Val_X, Train_Y, Val_Y = train_test_split(Train_X,Train_Y,test_size=0.15,random_state=11)

In [6]:
train_data_gen = ImageDataGenerator(rotation_range=30,
                                   width_shift_range=1,
                                    brightness_range=[0.8,1.2],
                                    zoom_range=[0.8,1.2],
                                    rescale=1/255
                                   )
val_data_gen = ImageDataGenerator(rescale=1/255)

test_data_gen = ImageDataGenerator(rescale=1/255)

In [7]:
set_seed(42)
random.seed(42)
np.random.seed(42)

val_data = val_data_gen.flow(Val_X,Val_Y,
                                   seed=42,shuffle=False)

test_data = test_data_gen.flow(Test_X,Test_Y,seed=42,shuffle=False)

In [8]:
from tensorflow import keras
loss_function = keras.losses.BinaryCrossentropy()
optimizer = Adam()

# Set Seed
random.seed(42)
set_seed(42)
np.random.seed(42)
  
# Define the model architecture
model = Sequential()
  
model.add(Conv2D(64, kernel_size=(3,3), input_shape=(128, 128, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())

model.add(Conv2D(128, kernel_size=(3,3), activation='relu', padding='same'))
model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))
model.add(BatchNormalization())
    
model.add(Conv2D(256, kernel_size=(3,3), activation='relu', padding='same'))
model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))
model.add(BatchNormalization())

model.add(Conv2D(512, kernel_size=(3,3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(1, activation='sigmoid'))


In [9]:
from keras.optimizers import RMSprop, SGD, Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [11]:
checkpoint = ModelCheckpoint('EmotionDetectionModel.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=25,
                          verbose=1,
                          restore_best_weights=True
                          )

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=3,
                              verbose=1,
                              min_delta=0.0001)

callbacks = [earlystop,checkpoint,reduce_lr]

opt = keras.optimizers.Adam(lr=0.001)

In [12]:
model.compile(loss='binary_crossentropy',
    optimizer=opt,
    metrics=['accuracy'])

In [13]:
history = model.fit(train_data_gen.flow(Train_X, Train_Y, seed=42),epochs=50, validation_data=val_data, callbacks=callbacks)

Epoch 1/50
80/80 [==============================] - 17s 156ms/step - loss: 1.6017 - accuracy: 0.5577 - val_loss: 1.0076 - val_accuracy: 0.5644

Epoch 00001: val_loss improved from inf to 1.00764, saving model to EmotionDetectionModel.h5
Epoch 2/50
80/80 [==============================] - 12s 144ms/step - loss: 1.0022 - accuracy: 0.6448 - val_loss: 4.9958 - val_accuracy: 0.5089

Epoch 00002: val_loss did not improve from 1.00764
Epoch 3/50
80/80 [==============================] - 12s 145ms/step - loss: 0.7358 - accuracy: 0.6665 - val_loss: 1.0234 - val_accuracy: 0.5089

Epoch 00003: val_loss did not improve from 1.00764
Epoch 4/50
80/80 [==============================] - 12s 147ms/step - loss: 0.5634 - accuracy: 0.7099 - val_loss: 1.1376 - val_accuracy: 0.5378
Restoring model weights from the end of the best epoch.

Epoch 00004: val_loss did not improve from 1.00764

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
Epoch 00004: early stopping


In [ ]:
# Generate generalization metrics
fig = px.line(
history.history, y=['loss', 'val_loss'],
labels={'index': 'epoch', 'value': 'val_loss'}, 
title='Training History')
fig.show()


# Generate generalization metrics
fig = px.line(
history.history, y=['accuracy', 'val_accuracy'],
labels={'index': 'epoch', 'value': 'accuracy'}, 
title='Training History')
fig.show()

In [ ]:
model.evaluate(test_data)